# 신경망 모델 구성하기 
- torch.nn: 신경망을 구성하는데 필요한 모든 구성 요소를 제공

In [1]:
# FashionMNIST 데이터셋의 이미지들을 분류하는 신경망을 구성
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


### 클래스 정의하기
- 신경망 모델을 nn.Module의 하위클래스로 정의
- \__init\__에서 신경망 계층 초기화
- nn.Module을 상속받은 모든 클래스는 forward메소드에 입력데이터에 대한 연산들을 구현

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__() # 상위클래스의 __init__ 오버라이딩
        self.flatten = nn.Flatten() # fc계층
        self.linear_relu_stack = nn.Sequential( # sequential모델 
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x): # 순전파
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
# NeuralNetwork의 instance생성 및 구조 출력
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
# 모델 실행
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits) 
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3])


### 모델 계층(Layer)
- nn.Flaten: flatten layer
- nn.Linear: linear transformation layer
- nn.ReLU: activation layer 
- nn.Sequential: sequential container
- nn.Softmax: probability layer

In [14]:
# 예제용 이미지 load
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [15]:
# nn.Flatten
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [17]:
# nn.Linear
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [18]:
# nn.ReLU
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1379, -0.0435,  0.1901, -0.0557,  0.0029, -0.5925,  0.5388,  0.5341,
          0.0657, -0.7460,  0.0411, -0.0038,  0.1444, -0.1524,  0.0174,  0.3736,
         -0.1742, -0.3983,  0.4670, -0.3246],
        [ 0.3739, -0.1151,  0.0431, -0.3800,  0.0138, -0.2340,  0.3977,  0.8193,
         -0.0431, -0.2369,  0.0411,  0.1651,  0.4614, -0.0993,  0.0791, -0.1140,
         -0.1732, -0.4640, -0.0224,  0.0967],
        [ 0.1773,  0.1942,  0.1600,  0.0840,  0.0202, -0.1066,  0.2846,  0.4060,
          0.0060, -0.4649, -0.0238,  0.3011,  0.3653, -0.0887, -0.1376,  0.3716,
          0.0737, -0.2242, -0.0047, -0.1151]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1379, 0.0000, 0.1901, 0.0000, 0.0029, 0.0000, 0.5388, 0.5341, 0.0657,
         0.0000, 0.0411, 0.0000, 0.1444, 0.0000, 0.0174, 0.3736, 0.0000, 0.0000,
         0.4670, 0.0000],
        [0.3739, 0.0000, 0.0431, 0.0000, 0.0138, 0.0000, 0.3977, 0.8193, 0.0000,
         0.0000, 0.0411, 0.1651, 0.4614, 0.0000, 0.07

In [19]:
# nn.Sequential
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [20]:
# nn.Softmax
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### 모델의 매개변수
- nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 parameter() 및 names_parameters()메소드로 모든 매개변수에 접근할 수 있다.

In [22]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters(): # name_parameters로 각 층의 파라미터 불러오기
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0208,  0.0357,  0.0186,  ...,  0.0131,  0.0310,  0.0301],
        [-0.0304,  0.0044, -0.0160,  ...,  0.0352,  0.0073,  0.0037]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0304, -0.0137], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0416, -0.0276, -0.0138,  ..., -0.0432, -0.0319,  0.0092],
        [-0.0328,  0.0133,  0.0435,  ..., -0.0158, -0.0182,  0.0157]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias 